# Flyff leveling bot

This notebooks is my personal test, playing object detection. Using opencv, I will create a simple leveling bot, that would find the nearest target and attack it. The bot is created for Flyff universe, and the imputs would have to be the `png file` of a target (I use target name snipets) and `png file` of character name. In order to swap from 1v1 mode, you will have to change from `doubleClick()` to `.press("key")`
## Import necessary Libraries

In [1]:
from re import template
import cv2 as cv
import pyautogui
from time import sleep
import numpy as np
import os
import math
import mss
import tempfile

In [2]:
# define constants ( target and main characted)
target_mob = 'banger.PNG'
avtr_name = "avtr_name.PNG"

## Define functions
### 1. Find all mobs in the screen

In [3]:
def findClickPositions(needle_img, haystack_img, threshold=0.5, debug_mode=None):
    # https://docs.opencv.org/4.2.0/d4/da8/group__imgcodecs.html
    # haystack_img = cv.imread(haystack_img_path, cv.IMREAD_UNCHANGED)
    # needle_img = cv.imread(needle_img_path, cv.IMREAD_UNCHANGED)
    # Save the dimensions of the needle image
    needle_w = needle_img.shape[1]
    needle_h = needle_img.shape[0]

    # There are 6 methods to choose from:
    # TM_CCOEFF, TM_CCOEFF_NORMED, TM_CCORR, TM_CCORR_NORMED, TM_SQDIFF, TM_SQDIFF_NORMED
    method = cv.TM_CCOEFF_NORMED
    result = cv.matchTemplate(
            image = haystack_img,
            templ=needle_img,
            method = cv.TM_CCOEFF_NORMED
        )

    # Get the all the positions from the match result that exceed our threshold
    locations = np.where(result >= threshold)
    locations = list(zip(*locations[::-1]))
    #print(locations)

    # You'll notice a lot of overlapping rectangles get drawn. We can eliminate those redundant
    # locations by using groupRectangles().
    # First we need to create the list of [x, y, w, h] rectangles
    rectangles = []
    for loc in locations:
        rect = [int(loc[0]), int(loc[1]), needle_w, needle_h]
        # Add every box to the list twice in order to retain single (non-overlapping) boxes
        rectangles.append(rect)
        rectangles.append(rect)
    # Apply group rectangles.
    # The groupThreshold parameter should usually be 1. If you put it at 0 then no grouping is
    # done. If you put it at 2 then an object needs at least 3 overlapping rectangles to appear
    # in the result. I've set eps to 0.5, which is:
    # "Relative difference between sides of the rectangles to merge them into a group."
    rectangles, weights = cv.groupRectangles(rectangles, groupThreshold=1, eps=0.5)
    points = []
    if len(rectangles):
        line_color = (0, 255, 0)
        line_type = cv.LINE_4
        marker_color = (255, 0, 255)
        marker_type = cv.MARKER_CROSS

        # Loop over all the rectangles
        for (x, y, w, h) in rectangles:

            # Determine the center position
            center_x = x + int(w/2)
            center_y = y + int(h/2) + 50
            # Save the points
            points.append((center_x, center_y))

            if debug_mode == 'rectangles':
                # Determine the box position
                top_left = (x, y)
                bottom_right = (x + w, y + h)
                # Draw the box
                cv.rectangle(haystack_img, top_left, bottom_right, color=line_color, 
                             lineType=line_type, thickness=2)
            elif debug_mode == 'points':
                # Draw the center point
                cv.drawMarker(haystack_img, (center_x, center_y), 
                              color=marker_color, markerType=marker_type, 
                              markerSize=40, thickness=2)

        if debug_mode:
            cv.imshow('Matches', haystack_img)
            cv.waitKey(0)
            #cv.imwrite('result_click_point.jpg', haystack_img)

    return points


### 2. Find Avatar position

In [4]:
def find_avatar_position(needle_img, haystack, threshold = 0.5):
    # needle_img = cv.imread(avatar, cv.IMREAD_UNCHANGED)
    # haystack_img = cv.imread(haystack, cv.IMREAD_UNCHANGED)

    needle_w = needle_img.shape[1]
    needle_h = needle_img.shape[0]
        # # There are 6 methods to choose from:
        # # TM_CCOEFF, TM_CCOEFF_NORMED, TM_CCORR, TM_CCORR_NORMED, TM_SQDIFF, TM_SQDIFF_NORMED
    result = cv.matchTemplate(haystack, needle_img, cv.TM_CCOEFF_NORMED)
    mi_val, ma_val, mi_loc, ma_loc = cv.minMaxLoc(result)
    # let's search for the needle and find the center location
    if ma_val >= threshold:
        top_left = ma_loc
        center_x = top_left[0] + int(needle_w/2)
        center_y = top_left[1] + int(needle_h/2)
        return np.array((center_x, center_y))
    else:
        pass


### 3. Find the closest target relative to the main character

In [5]:
# calculate distance and return the nearest point to the avatar
def distance(p1, p2):
    return math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)

def closest(pt, others):
    return min(others, key = lambda i: distance(pt, i))

### 4. Take Screenshots

In [6]:
def take_screenshot():
    try:
        os.remove('ssf.png')
    except Exception as e:
        pass
    temp_dir = tempfile.gettempdir()
    os.chdir(temp_dir)
    sleep(3)
    with mss.mss() as screenshot:
        screenshot.shot(output="ssf.png")
        hayhay = cv.imread("ssf.png", cv.IMREAD_UNCHANGED) 
            # cv.imshow('kaaaaaa', hayhay)
            # cv.waitKey(0)
        return hayhay

### 5. PyAutogui for auto clicking

In [7]:
def auto_click(target_mob, avtr_name, geimas):
    points = findClickPositions(target_mob, geimas , threshold=0.5)
    avatar_center = find_avatar_position(avtr_name, geimas, threshold = 0.5)
    click_pos = closest(avatar_center, points)
    return pyautogui.doubleClick(click_pos[0], click_pos[1])
     

### initialize the main loop

In [17]:
for i in range(3): ### checkup look ( test if it works)
    sleep(3)
    with mss.mss() as screenshot:
        hayhay = np.array(screenshot.grab(screenshot.monitors[0]))

    target_mob = cv.imread(target_mob, cv.IMREAD_UNCHANGED)
    avtr_name = cv.imread(avtr_name, cv.IMREAD_UNCHANGED)

    points = findClickPositions(target_mob, hayhay , threshold=0.7)
    avatar_center = find_avatar_position(avtr_name, hayhay, threshold = 0.7)
    try:
        click_pos = closest(avatar_center, points)
        pyautogui.doubleClick(click_pos[0], click_pos[1])
        key = cv.waitKey(1)
        if key == ord('q'):
            cv.destroyAllWindows()
            break
    except Exception as e:
        pass


In [ ]:
# future development (continous leveling)
# for multiple checks like this
            # checks = {
            #         "recieve.png": 'recieve',
            #         "next.png": 'next',...}

# need what is being searched for and haystackImage is where we are looking

# def detect(imgDir, haystackImage, confidence=0.85, cache=False):
#     if (pyautogui.locate(os.path.join('images', imgDir), haystackImage, confidence=confidence)) is not None:
#         return True
#     else:
#         return False

# from matplotlib import image
# hsImage = image.imread('images\\capture.jpeg')
# hsImage = hsImage[:,:,::-1] # convert RGB to BGR
# detect('needleImg.png', hsImage, cache=True) 